# Yearly Ridership Data Visualization And Analysis For The Taipei Metro System In Taiwan.

## Section 1. Aggregate Data
##### From Different Data Sources

### Steps
1. Retrieve coordinates from shapefiles
2. Check the data (look for missing data)
3. Add new station names and coordinates when needed (Enter missing data)
4. Retrieve ridership data from excel tables or CSV files
5. Aggregate data (i.e. daily to monthly, yearly...)
6. Put data together
7. Check the result
8. Export data to desired data format (such as JSON, GeoJSON...)


In [26]:
# import module
import geopandas as gpd
import pandas as pd


In [27]:
# read shapefile to extract coordinates
df_metro_shap = gpd.read_file('D:\上課資料\Geog778\Project\Data\Processed\MRT_TPE_Station_Modified\TpeMrtStations_TWD97_Unicode.shp')
# TWD97 => epsg=3824 (degree)
df_metro_shap_geo = df_metro_shap.to_crs(epsg=3824)

# create a coordinate dictionary
dic_coord= {}
stat_num = len(df_metro_shap_geo)
for i in range(stat_num):
    dic_coord[df_metro_shap_geo['EngName'][i]] = [df_metro_shap_geo['geometry'][i].x, df_metro_shap_geo['geometry'][i].y]

# add missing coordinates (Dingpu)
dic_coord['Dingpu'] = [121.41927, 24.95975]
# fix inaccurate coordinates
dic_coord['BL-Banqiao'] = [121.46242, 25.01409]
dic_coord['Y-Banqiao'] = [121.46469, 25.01554]
dic_coord['Xinpu Minsheng'] = [121.46697, 25.02630]
# 119 stations in total

In [28]:
# create year list
year_lst= list(range(2015,2023))
dic_sum = {}
year_num = len(year_lst)

# create station name list
station_df = pd.read_excel("D:/上課資料/Geog778/Project/Data/Processed/Ridership_{year}.xlsx".format(year= str(max(year_lst))), usecols="B:DP")
station_name = []
for col in station_df:
    station_name.append(col)

In [29]:
csv_lines = []
csvHeader = ['year', 'stationName', 'ridership', 'longitude', 'latitude']

# read all yearly ridership sum
for year in year_lst:
    df_sum = pd.read_excel("D:/上課資料/Geog778/Project/Data/Processed/Ridership_{year}.xlsx".format(year= str(year)), usecols="B:DP").sum()
    sum_lst = []
    for num in df_sum:
        sum_lst.append(round(num))
    dic_sum[str(year)]= sum_lst

    for (num, st_name, name) in zip(sum_lst, station_name, dic_coord):
        # row: year, stationName, ridership, longitude, latitude
        row = [year, st_name, num, dic_coord[st_name][0], dic_coord[st_name][1]]
        csv_lines.append(row)

In [30]:
# If want to export the data to CSV format, activate this cell

import csv 
# create a empty text file to receive data
with open("ridership.csv", 'w+', newline="") as f:
    writer = csv.writer(f)
    writer.writerow(csvHeader)
    writer.writerows(csv_lines)

In [31]:
# create a new dataframe to contain data
df_yearlyRid = pd.DataFrame()

# add station names
df_yearlyRid['stationName'] = station_name

# add ridership data
for year in year_lst:
    df_yearlyRid['yr'+str(year)] = dic_sum[str(year)] 

# add coordinates
name_num = len(station_name)
lng = []
lat = []
for i in range(name_num):
    lng.append(dic_coord[df_yearlyRid['stationName'][i]][0])
    lat.append(dic_coord[df_yearlyRid['stationName'][i]][1])
df_yearlyRid['longitude']=lng
df_yearlyRid['latitude']=lat
df_yearlyRid


,stationName,yr2015,yr2016,yr2017,yr2018,yr2019,yr2020,yr2021,yr2022,longitude,latitude
0,Songshan Airport,1996920,2092136,2019020,2033526,2088370,1445689,969124,1125624,121.551635,25.063104
1,Zhongshan Junior High School,5264138,5256347,5160555,5134302,5121762,4530029,3428641,3701648,121.544216,25.060806
2,Nanjing Fuxing,11111846,12013775,12601508,13322009,13782338,12528536,9553176,10749968,121.544039,25.052292
3,Zhongxiao Fuxing,18324235,18469890,17883808,17795928,17588313,14802750,11101659,12028310,121.543794,25.041602
4,Daan,7454570,7923790,8232428,8631916,8877145,7901826,5961757,6480952,121.543626,25.032821
...,...,...,...,...,...,...,...,...,...,...,...
114,Banxin,0,0,0,0,0,768557,718162,852545,121.472930,25.014003
115,Y-Banqiao,0,0,0,0,0,2599601,2197512,2616227,121.464690,25.015540
116,Xinpu Minsheng,0,0,0,0,0,709877,598946,675441,121.466970,25.026300
117,Xingfu,0,0,0,0,0,1066536,983128,1143045,121.459902,25.049784


In [32]:
# convert dataframe to GeoJSON
gdf_yearlyRid = gpd.GeoDataFrame(df_yearlyRid, geometry=gpd.points_from_xy(df_yearlyRid.longitude, df_yearlyRid.latitude))

# save as GeoJSON and export the data
yearlyRid_GeoJ= gdf_yearlyRid.to_file('D:/上課資料/Geog778/Project/Data/Processed/ridership.geojson', driver='GeoJSON')

c:\KC\anaconda3\envs\KC_Geog573\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## Section 2. Data Visualization with Pydeck
### Steps
1. Read the data
2. Calculate ridership for color ranking (High, Medium, Low)
3. Scaled down number range for visualization
4. Set initial view for the map
5. Create map layer
6. Customize the texts displayed (Clean up unnecessary content)
7. Create widget for controlling the map
8. Connect widget and the map
9. Display the map


In [33]:
# import module
import geopandas as gpd
import pandas as pd
import pydeck
import ipywidgets

In [34]:
# data
# tpeMRTridership = pd.DataFrame(csv_lines, columns=csvHeader)

# use this line to read data for mockup testing only
tpeMRTridership = pd.read_csv('ridership.csv')

# using the first and the third quantiles for ranking
quantiles = tpeMRTridership.ridership.quantile([0.25, 0.5, 0.75])
q_1 = quantiles[0.25]
q_3 = quantiles[0.75]

# scaled down the ridership for visualization
tpeMRTridership['scaled_ridership'] = tpeMRTridership['ridership']/100_000
df_leng = len(tpeMRTridership)
rank = []
color_set = []
for i in range(df_leng):
    if(tpeMRTridership['ridership'][i] >= q_3):
        color_set.append(0.9)
        rank.append('High')
    elif(tpeMRTridership['ridership'][i] >= q_1):
        color_set.append(0.5)
        rank.append('Medium')
    elif(tpeMRTridership['ridership'][i] ==0 ):
        color_set.append(0)
        rank.append('This station has not been built yet')
    else:
        color_set.append(0.3)
        rank.append('Low')
tpeMRTridership['colorRank'] =color_set
tpeMRTridership['rank'] =rank

# set initial view
view = pydeck.ViewState(latitude=25.05, longitude=121.50, pitch=30, zoom=9)

# layer
tpeMRTridership_layer = pydeck.Layer('ColumnLayer',
                             data=tpeMRTridership,
                             get_position=['longitude', 'latitude'],
                             get_elevation='scaled_ridership',
                             elevation_scale=100,
                             radius=300,
                             get_fill_color=['255', '255*colorRank', '200*colorRank', '150'],
                             pickable=True,
                             auto_highlight=True)

# Customize the texts displayed
tooltip = {
    'html': """
    <p><b>Name:</b> {stationName}</p>
    <p><b>Coordinate:</b> {longitude}, {latitude}</p>
    <p><b>Year:</b> {year}</p>
    <p><b>Ridership:</b> {ridership}</p>
    <p><b>Ridership Rank:</b> {rank}</p>
    """
}


# render map
# with no map_style, map goes to default
tpeMRTridership_layer_map = pydeck.Deck(layers=tpeMRTridership_layer, 
                               initial_view_state=view,
                               tooltip=tooltip
                               )

# widgets
slider = ipywidgets.IntSlider(value=2015, min=2015, max=2022, step=1)
play = ipywidgets.Play(value=2015, min=2015, max=2022, step=1, description='Press play', interval=1_000)
ipywidgets.jslink((play, 'value'), (slider, 'value'))
layout = ipywidgets.HBox([slider, play])

# widgets function
def update_plot(year):
    tpeMRTridership_layer.data = tpeMRTridership[tpeMRTridership['year'] == year]
    return tpeMRTridership_layer_map.update()

# interaction between widget and function
interact = ipywidgets.interactive_output(update_plot, {'year': slider})





In [35]:
# display the map and the widget
print("Legend")
print("Light Yellow: High")
print("Orange: Medium")
print("Red: Low")
display(layout, interact)
tpeMRTridership_layer_map.show()


Legend
Light Yellow: High
Orange: Medium
Red: Low


Output()

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"lat…